In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers , activations , models , preprocessing , utils
import pandas as pd

## Reading the Data: 


In [2]:
path = 'E:/Ai Notebooks/Chatbot-data/clean_conversation.txt'

In [3]:
input_text = []
target_text = []
with open(path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(600, len(lines) - 1)]:
    input_t = line.split('\t')[0]
    target_t = line.split('\t')[1]
    input_text.append(input_t)
    target_text.append(target_t)

In [4]:
len(input_text)

566

In [5]:
zipped_list = list(zip(input_text, target_text))
lines = pd.DataFrame(zipped_list, columns=['input', 'output'])

In [6]:
lines.head()

,input,output
0,What are your interests,I am interested in all kinds of things. We can...
1,What are your favorite subjects,"My favorite subjects include robotics, compute..."
2,What are your interests,"I am interested in a wide variety of topics, a..."
3,What is your number,I don't have any number
4,What is your number,23 skiddoo!


## Preparing input data for Encoder:

In [10]:
input_lines = list()
for line in lines.input:
    input_lines.append(line)
    
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(input_lines)
tokenized_input_lines = tokenizer.texts_to_sequences(input_lines)

len_list = list()
for token_line in tokenized_input_lines:
    len_list.append(len(token_line))
max_len = np.array(len_list).max()
print( 'Input max length is {}'.format( max_len ))

padded_input_lines = preprocessing.sequence.pad_sequences(tokenized_input_lines, maxlen=max_len, padding='post')
encoder_input_data = np.array(padded_input_lines)
print( 'Encoder input data shape -> {}'.format( encoder_input_data.shape ))

input_word_dict = tokenizer.word_index
num_input_tokens = len(input_word_dict) + 1 
print( 'Number of Input tokens = {}'.format( num_input_tokens))


Input max length is 22
Encoder input data shape -> (566, 22)
Number of Input tokens = 518


## Prepare input for Decoder: 


In [17]:
output_lines = list()
for line in lines.output:
    output_lines.append('<START> ' + line +  ' <END>')
    
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(output_lines)
tokenized_output_lines = tokenizer.texts_to_sequences(output_lines)

length_list = list()
for token_seq in tokenized_output_lines:
    length_list.append( len( token_seq ))
max_output_length = np.array( length_list ).max()
print( 'Output max length is {}'.format( max_output_length ))

padded_output_lines = preprocessing.sequence.pad_sequences(tokenized_output_lines, maxlen=max_output_length, padding='post')
decoder_input_data = np.array(padded_output_lines)
print( 'Decoder input data shape -> {}'.format( decoder_input_data.shape ))

output_word_dict = tokenizer.word_index
num_output_tokens = len(output_word_dict) + 1 
print( 'Number of Input tokens = {}'.format( num_output_tokens))

Output max length is 74
Decoder input data shape -> (566, 74)
Number of Input tokens = 1692


## Prepare target data for decoder:


In [20]:
decoder_target_data = list()
for token in tokenized_output_lines:
    decoder_target_data.append(token[1:])
    
padded_output_lines = preprocessing.sequence.pad_sequences(decoder_target_data, maxlen=max_output_length, padding='post')
onehot_output_lines = utils.to_categorical(padded_output_lines, num_output_tokens)
decoder_target_data = np.array(onehot_output_lines)
print( 'Decoder target data shape -> {}'.format( decoder_target_data.shape ))

Decoder target data shape -> (566, 74, 1692)


## Model Building:

In [23]:
encoder_inputs = tf.keras.layers.Input(shape=( None , ))
encoder_embedding = tf.keras.layers.Embedding( num_input_tokens, 256 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 256 , return_state=True , recurrent_dropout=0.2 , dropout=0.2 )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
decoder_embedding = tf.keras.layers.Embedding( num_output_tokens, 256 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 256 , return_state=True , return_sequences=True , recurrent_dropout=0.2 , dropout=0.2)
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( num_output_tokens , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy')

model.summary()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 256)    132608      input_2[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 256)    433152      input_3[0][0]                    


In [24]:
model.fit([encoder_input_data , decoder_input_data], decoder_target_data, batch_size=124, epochs=250) 
model.save( 'model.h5' ) 

Epoch 1/250
566/566 [==============================] - 17s 30ms/sample - loss: 1.4406
Epoch 2/250
566/566 [==============================] - 10s 18ms/sample - loss: 1.4348
Epoch 3/250
566/566 [==============================] - 11s 20ms/sample - loss: 1.3829
Epoch 4/250
566/566 [==============================] - 11s 20ms/sample - loss: 1.2243
Epoch 5/250
566/566 [==============================] - 13s 23ms/sample - loss: 1.1490
Epoch 6/250
566/566 [==============================] - 14s 24ms/sample - loss: 1.1212
Epoch 7/250
566/566 [==============================] - 15s 27ms/sample - loss: 1.1100
Epoch 8/250
566/566 [==============================] - 17s 30ms/sample - loss: 1.1035
Epoch 9/250
566/566 [==============================] - 16s 29ms/sample - loss: 1.0962
Epoch 10/250
566/566 [==============================] - 16s 27ms/sample - loss: 1.0905
Epoch 11/250
566/566 [==============================] - 14s 25ms/sample - loss: 1.0832
Epoch 12/250
566/566 [==============================

## Inference Model: 

In [25]:
def make_inference_model():
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    decoder_state_input_h = tf.keras.layers.Input(shape=(256,))
    decoder_state_input_c = tf.keras.layers.Input(shape=(256,))
    
    decoder_state_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding, decoder_state_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model([decoder_inputs] + decoder_state_inputs, [decoder_outputs] + decoder_states)
    return encoder_model, decoder_model

In [26]:
def str_to_token (sentence: str):
    words = sentence.lower().split()
    token_list = list()
    for word in words: 
        token_list.append(input_word_dict[word])
    return preprocessing.sequence.pad_sequences([token_list], maxlen=max_len, padding='post')


In [ ]:
enc_model , dec_model = make_inference_model()

enc_model.save( 'enc_model.h5' ) 
dec_model.save( 'dec_model.h5' ) 
model.save( 'model.h5' ) 

for epoch in range( encoder_input_data.shape[0] ):
    states_values = enc_model.predict( str_to_token( input( 'User: ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = output_word_dict['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in output_word_dict.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > max_output_length:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( "Bot:" +decoded_translation.replace(' end', '') )
    print()

User: hi
Bot: hello

User: how are you
Bot: i certainly am i shouldn't try so hard

User: what is ai
Bot: ai is the field of science which concerns itself with building hardware and software that replicates the functions of the human mind

